This version 2 incorporates the merchant cred quality (newly derived variable). At this point, this is specifically only to create cross tab of merchant_cred_quality x final risk class (no interrate (technically unchanged from version3) based on depth 4 only).

In [1]:
import pandas as pd
import numpy as np

In [2]:
base_df = pd.read_csv(r"C:\Users\emanlangit\Documents\2025\03_ASBKH\03 Merchant Analysis\02 ASBC Analysis 05132025\Data\Base_Data_05132025.csv")
df = base_df.copy()

C:\Users\emanlangit\AppData\Local\Temp\ipykernel_5072\3118858516.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  base_df = pd.read_csv(r"C:\Users\emanlangit\Documents\2025\03_ASBKH\03 Merchant Analysis\02 ASBC Analysis 05132025\Data\Base_Data_05132025.csv")


In [3]:
df.shape

(444001, 31)

In [4]:
df['appdate'] = df['appdate'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors = 'coerce'))
df['agreementdate'] = df['agreementdate'].apply(lambda x: pd.to_datetime(x, format='%Y/%m/%d', errors = 'coerce'))

In [5]:
df['D3_within8m'] = df['agreementdate_months_elapsed_valid'].apply(lambda x: 1 if 3 <= x <= 8 else 0)

# Function to add tags
def add_tags(date):
    if date >= pd.Timestamp('2023-01-01') and date <= pd.Timestamp('2023-07-31'):
        return 'Interval_1'
    elif date >= pd.Timestamp('2024-01-01') and date <= pd.Timestamp('2024-07-31'):
        return 'Interval_2'
    else:
        return 'Other'

# Apply the function to the Date column
df['Interval_Tag'] = df['agreementdate'].apply(add_tags)

In [6]:
#reset_df = df.copy(deep=True)
#back_up = reset_df.copy(deep=True)

In [ ]:
df = back_up.copy()

### Incorporate merchant cred quality variable

In [7]:
merchant_cred_qual_df = pd.read_csv(r"C:\Users\emanlangit\Documents\2025\03_ASBKH\03 Merchant Analysis\02 ASBC Analysis 05132025\Notebooks\MHP_2023_overall_credit_quality.csv")
merchant_cred_qual_df['credit_quality_ratio'] = merchant_cred_qual_df.credit_quality_ratio.fillna(-999)
merged_df = pd.merge(df, merchant_cred_qual_df, on ='storecd', how='inner')
df = merged_df.copy()

In [8]:
merchants_AD_2023 = 3710 / 8796
lowrisk_upperbound = merchants_AD_2023*(1/3) #this is different from previous formula which used: average default * 2/3
middlerisk_upperbound = merchants_AD_2023*(2/3) #his is different from previous formula which used: average default * 2/3 *2
middlehighrisk_upperbound = merchants_AD_2023*(2) #same with past formula
#salary ok
def credit_quality_ratio_bin(x):
    try:
        y = float(x)
        bins = [
            (middlehighrisk_upperbound, 'High Risk'),
            (middlerisk_upperbound, 'Middle High Risk'),
            (lowrisk_upperbound, 'Middle Risk'),
            (-1, 'Low Risk'),
            (-1000, 'No data available')     
        ]
        
        for threshold, label in bins:
            if y > threshold:
                return label
    except ValueError:
        return x

df.loc[:, 'merchant_cred_quality'] = df['credit_quality_ratio'].apply(credit_quality_ratio_bin)

# High Risk Branch Grouping

In [9]:
df['branchcode'] = df['branchcode'].astype(float)
df['branchcode'] = df['branchcode'].fillna(-999)
df['branchcode'] = df['branchcode'].astype(int)
df['branchcode'] = df['branchcode'].astype(str)

In [10]:
#branchcode
df['branchcode_grp'] = np.where(
    (df['branchcode'] == '10') | (df['branchcode'] == '15')
    | (df['branchcode'] == '9'),
    'HighRisk_branch', df['branchcode'])

df['branchcode_grp'] = np.where(
    (df['branchcode_grp'] != 'HighRisk_branch')
    , 'NonHighRisk_branch', df['branchcode_grp'])

# Prep Dataframes
## Divide Dataframes between High Risk and Non-High Risk

In [29]:
loantype_filter = df.loantype == 'MHP'
judgement_result_filter = df.judgement_result == 'APPROVED'
Interval1_Tag_filter = df.Interval_Tag == 'Interval_1'
Interval2_Tag_filter = df.Interval_Tag == 'Interval_2'
HighRiskBranch_filter = df.branchcode_grp == 'HighRisk_branch'
NonHighRiskBranch_filter = df.branchcode_grp == 'NonHighRisk_branch'

working_df_HighRiskBranch_filters = loantype_filter & judgement_result_filter & Interval2_Tag_filter & HighRiskBranch_filter
working_df_NonHighRiskBranch_filters = loantype_filter & judgement_result_filter & Interval2_Tag_filter & NonHighRiskBranch_filter

working_df1 = df[working_df_HighRiskBranch_filters]

working_df2 = df[working_df_NonHighRiskBranch_filters]

In [30]:
print(working_df1.shape)
print(working_df2.shape)

(3156, 36)
(4309, 36)


In [31]:
working_df2.D3_within8m.value_counts()

D3_within8m
0    4014
1     295
Name: count, dtype: int64

In [32]:
working_df1.columns

Index(['customercd', 'agreementcd', 'loantype', 'Risk_Grade', 'storecd',
       'currency', 'customertype', 'companytype', 'financeprice',
       'financeprice_USD', 'typeofresident', 'salary', 'livingwith',
       'agreementdate', 'judgement_result', 'interrate', 'period',
       'D3_YRMTH_valid', 'livingperiod_valid', 'workingperiod_valid',
       'age_valid', 'gender', 'maritalstatus', 'education', 'appdate',
       'appcd_months_elapsed_valid', 'agreementdate_months_elapsed_valid',
       'appcd', 'agreementcd_original', 'branchcode', 'position',
       'D3_within8m', 'Interval_Tag', 'credit_quality_ratio',
       'merchant_cred_quality', 'branchcode_grp'],
      dtype='object')

In [33]:
working_df1 = working_df1.rename(columns={'livingperiod_valid': 'livingperiod', 'workingperiod_valid': 'workingperiod',
                                         'age_valid':'age'})
working_df2 = working_df2.rename(columns={'livingperiod_valid': 'livingperiod', 'workingperiod_valid': 'workingperiod',
                                         'age_valid':'age'})

In [34]:
necessary_cols = ['customertype','companytype','financeprice_USD','typeofresident', 'salary', 'livingwith',
                 'interrate', 'period','livingperiod', 'workingperiod','age','gender',
                 'maritalstatus', 'education','position','D3_within8m','branchcode','credit_quality_ratio',
                 'merchant_cred_quality']
working_df1 = working_df1[necessary_cols]
working_df2 = working_df2[necessary_cols]

# Supporting EDA

In [35]:
eda_working_df1 = working_df1.copy()
eda_working_df2 = working_df2.copy()

In [15]:
#financeprice ok
def financeprice_bin(x):
    try:
        y = float(x)
        bins = [
            (3000, '3000 above'),
            (2700, '2701-3000'),
            (2400, '2401-2700'),
            (2100, '2101-2400'),
            (1800, '1801-2100'),
            (1500, '1501-1800'),
            (-1, '1500 below')
        ]
        
        for threshold, label in bins:
            if y > threshold:
                return label
    except ValueError:
        return x

eda_working_df1.loc[:, 'financeprice_grp'] = eda_working_df1['financeprice_USD'].apply(financeprice_bin)
eda_working_df2.loc[:, 'financeprice_grp'] = eda_working_df2['financeprice_USD'].apply(financeprice_bin)

#salary ok
def salary_bin(x):
    try:
        y = float(x)
        bins = [
            (600, '600 above'),
            (500, '501-600'),
            (400, '401-500'),
            (300, '301-400'),
            (-1, '300 below')
        ]
        
        for threshold, label in bins:
            if y > threshold:
                return label
    except ValueError:
        return x

eda_working_df1.loc[:, 'salary_grp'] = eda_working_df1['salary'].apply(salary_bin)
eda_working_df2.loc[:, 'salary_grp'] = eda_working_df2['salary'].apply(salary_bin)

#livingperiod ok
def livingperiod_bin(x):
    try:
        y = float(x)
        bins = [
            (27, '28 above'),
            (17, '18-27'),
            (7, '8-17'),
            (6, '7'),
            (5, '6'),
            (4, '5'),
            (3, '4'),
            (2, '3'),
            (1, '2'),
            (0, '1'),
            (-1, '0')
        ]
        
        for threshold, label in bins:
            if y > threshold:
                return label
    except ValueError:
        return x

eda_working_df1.loc[:, 'livingperiod_grp'] = eda_working_df1['livingperiod'].apply(livingperiod_bin)
eda_working_df2.loc[:, 'livingperiod_grp'] = eda_working_df2['livingperiod'].apply(livingperiod_bin)

#workingperiod ok
def workingperiod_bin(x):
    try:
        y = float(x)
        bins = [
            (7, '8 above'),
            (6, '7'),
            (5, '6'),
            (4, '5'),
            (3, '4'),
            (2, '3'),
            (1, '2'),
            (0, '1'),
            (-1, '0')
        ]
        
        for threshold, label in bins:
            if y > threshold:
                return label
    except ValueError:
        return x

eda_working_df1.loc[:, 'workingperiod_grp'] = eda_working_df1['workingperiod'].apply(workingperiod_bin)
eda_working_df2.loc[:, 'workingperiod_grp'] = eda_working_df2['workingperiod'].apply(workingperiod_bin)

#age 
def age_bin(x):
    try:
        y = float(x)
        bins = [
            (40, '40 above'),
            (35, '36 to 40'),
            (30, '31 to 35'),
            (25, '26 to 30'),
            (20, '21 to 25'),
            (-1, '21 below')
        ]
        
        for threshold, label in bins:
            if y > threshold:
                return label
    except ValueError:
        return x

eda_working_df1.loc[:, 'age_grp'] = eda_working_df1['age'].apply(age_bin)
eda_working_df2.loc[:, 'age_grp'] = eda_working_df2['age'].apply(age_bin)

In [16]:
# Convert the companytype column to categorical type
eda_working_df1['companytype'] = eda_working_df1['companytype'].astype('category')
# Define all possible categories
all_categories_companytype = ['PRIVATE COMPANY (LESS 50 STAFFS)', 'NGO', 'SELF-EMPLOYMENT',
       'FACTORY', 'PRIVATE COMPANY (50 STAFFS UP)', 'GOVERNMENT OFFICER',
       'STATE ENTERPRISE', 'OTHER', 'SPECIALIST (DOCTOR, LAWYER...ETC)',
       'EDUCATION', 'AGRICULTURE', 'BUSINESS OWNER (COMPANY)',
       'LISTED COMPANY']
# Set the categories
eda_working_df1['companytype'] = eda_working_df1['companytype'].cat.set_categories(all_categories_companytype)

# Convert the salary_grp column to categorical type
eda_working_df1['salary_grp'] = eda_working_df1['salary_grp'].astype('category')
# Define all possible categories
all_categories_salary_grp = ['300 below', '301-400', '401-500', '501-600','600 above']
# Set the categories
eda_working_df1['salary_grp'] = eda_working_df1['salary_grp'].cat.set_categories(all_categories_salary_grp)

# Convert the financeprice_grp column to categorical type
eda_working_df1['financeprice_grp'] = eda_working_df1['financeprice_grp'].astype('category')
# Define all possible categories
all_categories_financeprice_grp = ['1500 below', '1501-1800', '1801-2100', '2101-2400','2401-2700',
                                  '2701-3000','3000 above']
# Set the categories
eda_working_df1['financeprice_grp'] = eda_working_df1['financeprice_grp'].cat.set_categories(all_categories_financeprice_grp)

# Convert the workingperiod_grp column to categorical type
eda_working_df1['workingperiod_grp'] = eda_working_df1['workingperiod_grp'].astype('category')
# Define all possible categories
all_categories_workingperiod_grp = ['0', '1', '2', '3', '4', '5', '6','7','8 above']
# Set the categories
eda_working_df1['workingperiod_grp'] = eda_working_df1['workingperiod_grp'].cat.set_categories(all_categories_workingperiod_grp)

# Convert the livingperiod_grp column to categorical type
eda_working_df1['livingperiod_grp'] = eda_working_df1['livingperiod_grp'].astype('category')
# Define all possible categories
all_categories_livingperiod_grp = ['0', '1', '2', '3', '4', '5', '6','7','8-17','18-27','28 above']
# Set the categories
eda_working_df1['livingperiod_grp'] = eda_working_df1['livingperiod_grp'].cat.set_categories(all_categories_livingperiod_grp)

# Convert the age_grp column to categorical type
eda_working_df1['age_grp'] = eda_working_df1['age_grp'].astype('category')
# Define all possible categories
all_categories_age_grp = ['21 below', '21 to 25', '26 to 30', '31 to 35', '36 to 40', '40 above']
# Set the categories
eda_working_df1['age_grp'] = eda_working_df1['age_grp'].cat.set_categories(all_categories_age_grp)

In [17]:

# Convert the companytype column to categorical type
eda_working_df2['companytype'] = eda_working_df2['companytype'].astype('category')
# Define all possible categories
all_categories_companytype = ['PRIVATE COMPANY (LESS 50 STAFFS)', 'NGO', 'SELF-EMPLOYMENT',
       'FACTORY', 'PRIVATE COMPANY (50 STAFFS UP)', 'GOVERNMENT OFFICER',
       'STATE ENTERPRISE', 'OTHER', 'SPECIALIST (DOCTOR, LAWYER...ETC)',
       'EDUCATION', 'AGRICULTURE', 'BUSINESS OWNER (COMPANY)',
       'LISTED COMPANY']
# Set the categories
eda_working_df2['companytype'] = eda_working_df2['companytype'].cat.set_categories(all_categories_companytype)

# Convert the salary_grp column to categorical type
eda_working_df2['salary_grp'] = eda_working_df2['salary_grp'].astype('category')
# Define all possible categories
all_categories_salary_grp = ['300 below', '301-400', '401-500', '501-600','600 above']
# Set the categories
eda_working_df2['salary_grp'] = eda_working_df2['salary_grp'].cat.set_categories(all_categories_salary_grp)

# Convert the financeprice_grp column to categorical type
eda_working_df2['financeprice_grp'] = eda_working_df2['financeprice_grp'].astype('category')
# Define all possible categories
all_categories_financeprice_grp = ['1500 below', '1501-1800', '1801-2100', '2101-2400','2401-2700',
                                  '2701-3000','3000 above']
# Set the categories
eda_working_df2['financeprice_grp'] = eda_working_df2['financeprice_grp'].cat.set_categories(all_categories_financeprice_grp)

# Convert the workingperiod_grp column to categorical type
eda_working_df2['workingperiod_grp'] = eda_working_df2['workingperiod_grp'].astype('category')
# Define all possible categories
all_categories_workingperiod_grp = ['0', '1', '2', '3', '4', '5', '6','7','8 above']
# Set the categories
eda_working_df2['workingperiod_grp'] = eda_working_df2['workingperiod_grp'].cat.set_categories(all_categories_workingperiod_grp)

# Convert the livingperiod_grp column to categorical type
eda_working_df2['livingperiod_grp'] = eda_working_df2['livingperiod_grp'].astype('category')
# Define all possible categories
all_categories_livingperiod_grp = ['0', '1', '2', '3', '4', '5', '6','7','8-17','18-27','28 above']
# Set the categories
eda_working_df2['livingperiod_grp'] = eda_working_df2['livingperiod_grp'].cat.set_categories(all_categories_livingperiod_grp)

# Convert the age_grp column to categorical type
eda_working_df2['age_grp'] = eda_working_df2['age_grp'].astype('category')
# Define all possible categories
all_categories_age_grp = ['21 below', '21 to 25', '26 to 30', '31 to 35', '36 to 40', '40 above']
# Set the categories
eda_working_df2['age_grp'] = eda_working_df2['age_grp'].cat.set_categories(all_categories_age_grp)

## Prep DF for defaults

In [18]:
eda_working_df1_default_filter = eda_working_df1.D3_within8m == 1
eda_working_df1_default = eda_working_df1[eda_working_df1_default_filter]

eda_working_df2_default_filter = eda_working_df2.D3_within8m == 1
eda_working_df2_default = eda_working_df2[eda_working_df2_default_filter]

eda_working_df1_nondefault_filter = eda_working_df1.D3_within8m == 0
eda_working_df1_nondefault = eda_working_df1[eda_working_df1_nondefault_filter]

eda_working_df2_nondefault_filter = eda_working_df2.D3_within8m == 0
eda_working_df2_nondefault = eda_working_df2[eda_working_df2_nondefault_filter]

## Counts

In [112]:
#combine all results in one workbook
dist_categ_list = ['customertype','companytype','typeofresident',
                  'livingwith','gender','maritalstatus','education', 'position',
                  'salary_grp','financeprice_grp','livingperiod_grp','workingperiod_grp',
                  'age_grp']

def dist_categ(variable):
    t1_nondefault = eda_working_df1_nondefault[variable].value_counts().sort_index()
    t2_nondefault = eda_working_df2_nondefault[variable].value_counts().sort_index()
    t3_default = eda_working_df1_default[variable].value_counts().sort_index()
    t4_default = eda_working_df2_default[variable].value_counts().sort_index()
    
    sheetname = f'{variable}'
    mix_df = pd.DataFrame({
    'HighRiskBr Nondefault': t1_nondefault, 
    'NonHighRiskBr Nondefault': t2_nondefault,
    'HighRiskBr Default Counts': t3_default,
    'NonHighRiskBr Default': t4_default,
})
    mix_df.to_excel(writer,sheet_name=sheetname, index = True)

def num_categ(variable):
    t1_nondefault = eda_working_df1_nondefault[variable].astype(float).value_counts().sort_index()
    t2_nondefault = eda_working_df2_nondefault[variable].astype(float).value_counts().sort_index()
    t3_default = eda_working_df1_default[variable].astype(float).value_counts().sort_index()
    t4_default = eda_working_df2_default[variable].astype(float).value_counts().sort_index()
    
    sheetname = f'{variable}'
    mix_df = pd.DataFrame({
    'HighRiskBr Nondefault': t1_nondefault, 
    'NonHighRiskBr Nondefault': t2_nondefault,
    'HighRiskBr Default Counts': t3_default,
    'NonHighRiskBr Default': t4_default,
})
    mix_df.to_excel(writer,sheet_name=sheetname, index = True)
    
with pd.ExcelWriter('Supporting_EDA_MHP_2024_HRB_NHRBTemp.xlsx', engine='openpyxl') as writer:
    for i in range (0,len(dist_categ_list)):
        dist_categ(dist_categ_list[i])
    num_categ('period')
    num_categ('interrate')

# Tabulation and Others

 ## Assigning Risk Classes v1 (Using Interval 2 average default rate)

### High Risk Branches

In [36]:
try_df = eda_working_df1.copy()

In [37]:
#code here
HRB_conditions = [
    (try_df['period'] > 30) & (try_df['workingperiod'] <= 0.5) & (try_df['age'] <= 33.5)
    & (try_df['financeprice_USD'] <= 1765),
    (try_df['period'] > 30) & (try_df['workingperiod'] <= 0.5) & (try_df['age'] <= 33.5)
    & (try_df['financeprice_USD'] > 1765),
    (try_df['period'] > 30) & (try_df['workingperiod'] > 0.5) & (try_df['livingperiod'] <= 4.5)
    & (try_df['age'] <= 24.5),
    (try_df['period'] > 30) & (try_df['workingperiod'] <= 0.5) & (try_df['age'] > 33.5)
    & (try_df['education'] != 'HIGH SCHOOL'),
    (try_df['period'] <= 30) & (try_df['financeprice_USD'] <= 2115),
    (try_df['period'] > 30) & (try_df['workingperiod'] > 0.5) & (try_df['livingperiod'] <= 4.5)
    & (try_df['age'] > 24.5),
    (try_df['period'] > 30) & (try_df['workingperiod'] > 0.5) & (try_df['livingperiod'] > 4.5)
    & (try_df['livingwith'] == 'PARENTS'),
    (try_df['period'] <= 30) & (try_df['financeprice_USD'] > 2115) & (try_df['salary'] > 489),
    (try_df['period'] > 30) & (try_df['workingperiod'] <= 0.5) & (try_df['age'] > 33.5)
    & (try_df['education'] == 'HIGH SCHOOL'),
    (try_df['period'] <= 30) & (try_df['financeprice_USD'] > 2115) & (try_df['salary'] <= 489)
    & (try_df['gender'] == 'M'),
    (try_df['period'] > 30) & (try_df['workingperiod'] > 0.5) & (try_df['livingperiod'] > 4.5)
    & (try_df['livingwith'] != 'PARENTS'),
    (try_df['period'] <= 30) & (try_df['financeprice_USD'] > 2115) & (try_df['salary'] <= 489)
    & (try_df['gender'] != 'M'),
]

HRB_risk = ['Very High Risk', 'High Risk','High Risk','High Risk',
            'Middle Risk','Middle Risk','Middle Risk','Middle Risk',
            'Low Risk','Low Risk','Low Risk','Low Risk'
           ]

HRB_path = ['Path 1', 'Path 2','Path 3','Path 4',
            'Path 5','Path 6','Path 7','Path 8',
            'Path 9','Path 10','Path 11','Path 12'
           ]

try_df['Risk_Grade'] = np.select(HRB_conditions, HRB_risk, default='Other')
try_df['Risk_Path'] = np.select(HRB_conditions, HRB_path, default='Other')
#try_df.to_excel('HRB_scored_v1.xlsx', index = True)

eda_working_df1_risk_assigned = try_df.copy(deep=True)

In [38]:
#Checking if correct assignment N
try_df.Risk_Path.value_counts().sort_index()

Risk_Path
Path 1       67
Path 10     181
Path 11     249
Path 12     122
Path 2     1178
Path 3      434
Path 4      101
Path 5      119
Path 6      344
Path 7      207
Path 8       89
Path 9       65
Name: count, dtype: int64

In [39]:
try_df.Risk_Grade.value_counts().sort_index()

Risk_Grade
High Risk         1713
Low Risk           617
Middle Risk        759
Very High Risk      67
Name: count, dtype: int64

### Non High Risk Branches

In [40]:
try_df = eda_working_df2.copy()

In [41]:
try_df.companytype.unique()

array(['PRIVATE COMPANY (50 STAFFS UP)', 'SELF-EMPLOYMENT',
       'PRIVATE COMPANY (LESS 50 STAFFS)', 'FACTORY', 'EDUCATION',
       'GOVERNMENT OFFICER', 'OTHER', 'NGO', 'STATE ENTERPRISE',
       'AGRICULTURE', 'SPECIALIST (DOCTOR, LAWYER...ETC)',
       'LISTED COMPANY', 'BUSINESS OWNER (COMPANY)'], dtype=object)

In [42]:
#code here
NHRB_conditions = [
    (try_df['salary'] <= 325) & (try_df['gender'] == 'M') & (try_df['livingperiod'] <= 1.5)
    & (try_df['financeprice_USD'] > 2585),
    (try_df['salary'] <= 325) & (try_df['gender'] == 'M') & (try_df['livingperiod'] <= 1.5)
    & (try_df['financeprice_USD'] <= 2585),
    (try_df['salary'] <= 325) & (try_df['gender'] != 'M') & (try_df['position'] == 'OWNER'),
    (try_df['salary'] <= 325) & (try_df['gender'] == 'M') & (try_df['livingperiod'] > 1.5)
    & (try_df['livingwith'] != 'SPOUSE'),
    (try_df['salary'] > 325) & (try_df['period'] > 30) & (try_df['livingperiod'] <= 31.5)
    & (try_df['salary'] <= 402.585),
    (try_df['salary'] <= 325) & (try_df['gender'] == 'M') & (try_df['livingperiod'] > 1.5)
    & (try_df['livingwith'] == 'SPOUSE'),
    (try_df['salary'] > 325) & (try_df['period'] <= 30) & (try_df['livingperiod'] <= 1.5),
    (try_df['salary'] <= 325) & (try_df['gender'] != 'M') & (try_df['position'] != 'OWNER')
    & (try_df['salary'] > 199),
    (try_df['salary'] > 325) & (try_df['period'] > 30) & (try_df['livingperiod'] <= 31.5)
    & (try_df['salary'] > 402.585),
    (try_df['salary'] > 325) & (try_df['period'] <= 30) & (try_df['livingperiod'] > 1.5)
    & (try_df['companytype'] == 'PRIVATE COMPANY (LESS 50 STAFFS)'),
    (try_df['salary'] <= 325) & (try_df['gender'] != 'M') & (try_df['position'] != 'OWNER')
    & (try_df['salary'] <= 199),
    (try_df['salary'] > 325) & (try_df['period'] <= 30) & (try_df['livingperiod'] > 1.5)
    & (try_df['companytype'] != 'PRIVATE COMPANY (LESS 50 STAFFS)'),
    (try_df['salary'] > 325) & (try_df['period'] > 30) & (try_df['livingperiod'] > 31.5)    
]

NHRB_risk = ['Very High Risk','High Risk','Middle Risk','Middle Risk',
             'Middle Risk','Low Risk','Low Risk','Low Risk',
             'Low Risk','Low Risk','Low Risk','Low Risk',
             'Low Risk']

NHRB_path = ['Path 1', 'Path 2','Path 3','Path 4',
            'Path 5','Path 6','Path 7','Path 8',
            'Path 9','Path 10','Path 11','Path 12',
             'Path 13'
           ]

try_df['Risk_Grade'] = np.select(NHRB_conditions, NHRB_risk, default='Other')
try_df['Risk_Path'] = np.select(NHRB_conditions, NHRB_path, default='Other')
#try_df.to_excel('NHRB_scored_v1.xlsx', index = True)

eda_working_df2_risk_assigned = try_df.copy(deep=True)

In [43]:
eda_working_df2_risk_assigned.Risk_Grade.value_counts().sort_index()

Risk_Grade
High Risk          311
Low Risk          2578
Middle Risk       1331
Very High Risk      89
Name: count, dtype: int64

## Cross Tabulation

In [44]:
df_graded = pd.concat([eda_working_df1_risk_assigned,eda_working_df2_risk_assigned], axis=0)

In [45]:
#branchcode
df_graded['branchcode_grp'] = np.where(
    (df_graded['branchcode'] == '9') | (df_graded['branchcode'] == '10')
    | (df_graded['branchcode'] == '15'),
    'HighRisk_branch', df_graded['branchcode'])

df_graded['branchcode_grp'] = np.where(
    (df_graded['branchcode_grp'] != 'HighRisk_branch')
    , 'NonHighRisk_branch', df_graded['branchcode_grp'])

In [46]:
df_graded.columns

Index(['customertype', 'companytype', 'financeprice_USD', 'typeofresident',
       'salary', 'livingwith', 'interrate', 'period', 'livingperiod',
       'workingperiod', 'age', 'gender', 'maritalstatus', 'education',
       'position', 'D3_within8m', 'branchcode', 'credit_quality_ratio',
       'merchant_cred_quality', 'Risk_Grade', 'Risk_Path', 'branchcode_grp'],
      dtype='object')

In [47]:
df_graded['branchcode'] = df_graded['branchcode'].astype(int)

In [48]:
HRB_df = df_graded[df_graded.branchcode_grp == 'HighRisk_branch']
HRB_df_default = df_graded[(df_graded.branchcode_grp == 'HighRisk_branch') & (df_graded.D3_within8m == 1)]
print(HRB_df.shape)
print(HRB_df_default.shape)

NHRB_df = df_graded[df_graded.branchcode_grp == 'NonHighRisk_branch']
NHRB_df_default = df_graded[(df_graded.branchcode_grp == 'NonHighRisk_branch') & (df_graded.D3_within8m == 1)]
print(NHRB_df.shape)
print(NHRB_df_default.shape)

(3156, 22)
(433, 22)
(4309, 22)
(295, 22)


#### High Risk Branches

In [49]:
var_1 = 'merchant_cred_quality'
var_2 = 'Risk_Grade'
HRB_cross_all = pd.crosstab(HRB_df[var_1], HRB_df[var_2], dropna=False)
HRB_cross_all.sort_index()

Risk_Grade,High Risk,Low Risk,Middle Risk,Very High Risk
merchant_cred_quality,,,,
High Risk,72,16,24,4
Low Risk,61,19,30,4
Middle High Risk,1429,522,628,51
Middle Risk,55,22,25,0
No data available,96,38,52,8


In [50]:
var_1 = 'merchant_cred_quality'
var_2 = 'Risk_Grade'
HRB_cross_default = pd.crosstab(HRB_df_default[var_1], HRB_df_default[var_2], dropna=False)
HRB_cross_default.sort_index()

Risk_Grade,High Risk,Low Risk,Middle Risk,Very High Risk
merchant_cred_quality,,,,
High Risk,6,1,2,3
Low Risk,14,0,0,1
Middle High Risk,264,19,67,15
Middle Risk,8,0,2,0
No data available,20,3,7,1


In [51]:
with pd.ExcelWriter('HRB_crosstab temp.xlsx', engine='openpyxl') as writer:
    HRB_cross_all.to_excel(writer,sheet_name='all count', index = True)
    HRB_cross_default.to_excel(writer,sheet_name='default', index = True)

#### Non High Risk Branches

In [52]:
var_1 = 'merchant_cred_quality'
var_2 = 'Risk_Grade'
NHRB_cross_all = pd.crosstab(NHRB_df[var_1], NHRB_df[var_2], dropna=False)

In [53]:
var_1 = 'merchant_cred_quality'
var_2 = 'Risk_Grade'
NHRB_cross_default = pd.crosstab(NHRB_df_default[var_1], NHRB_df_default[var_2], dropna=False)

In [54]:
with pd.ExcelWriter('NHRB_crosstab temp.xlsx', engine='openpyxl') as writer:
    NHRB_cross_all.to_excel(writer,sheet_name='all count', index = True)
    NHRB_cross_default.to_excel(writer,sheet_name='default', index = True)

 ## Assigning Risk Classes v2 (Using NHRB average default rate)

In [116]:
try_df = eda_working_df1.copy()

#code here
HRB_conditions = [
    (try_df['period'] > 30) & (try_df['workingperiod'] <= 0.5) & (try_df['age'] <= 33.5)
    & (try_df['financeprice_USD'] <= 1765),
    (try_df['period'] > 30) & (try_df['workingperiod'] <= 0.5) & (try_df['age'] <= 33.5)
    & (try_df['financeprice_USD'] > 1765),
    (try_df['period'] > 30) & (try_df['workingperiod'] > 0.5) & (try_df['livingperiod'] <= 4.5)
    & (try_df['age'] <= 24.5),
    (try_df['period'] > 30) & (try_df['workingperiod'] <= 0.5) & (try_df['age'] > 33.5)
    & (try_df['education'] != 'HIGH SCHOOL'),
    (try_df['period'] <= 30) & (try_df['financeprice_USD'] <= 2115),
    (try_df['period'] > 30) & (try_df['workingperiod'] > 0.5) & (try_df['livingperiod'] <= 4.5)
    & (try_df['age'] > 24.5),
    (try_df['period'] > 30) & (try_df['workingperiod'] > 0.5) & (try_df['livingperiod'] > 4.5)
    & (try_df['livingwith'] == 'PARENTS'),
    (try_df['period'] <= 30) & (try_df['financeprice_USD'] > 2115) & (try_df['salary'] > 489),
    (try_df['period'] > 30) & (try_df['workingperiod'] <= 0.5) & (try_df['age'] > 33.5)
    & (try_df['education'] == 'HIGH SCHOOL'),
    (try_df['period'] <= 30) & (try_df['financeprice_USD'] > 2115) & (try_df['salary'] <= 489)
    & (try_df['gender'] == 'M'),
    (try_df['period'] > 30) & (try_df['workingperiod'] > 0.5) & (try_df['livingperiod'] > 4.5)
    & (try_df['livingwith'] != 'PARENTS'),
    (try_df['period'] <= 30) & (try_df['financeprice_USD'] > 2115) & (try_df['salary'] <= 489)
    & (try_df['gender'] != 'M'),
]

HRB_risk = ['Very High Risk', 'Very High Risk','Very High Risk','Very High Risk',
            'High Risk','High Risk','High Risk','Middle Risk',
            'Middle Risk','Low Risk','Low Risk','Low Risk'
           ]

HRB_path = ['Path 1', 'Path 2','Path 3','Path 4',
            'Path 5','Path 6','Path 7','Path 8',
            'Path 9','Path 10','Path 11','Path 12'
           ]

try_df['Risk_Grade'] = np.select(HRB_conditions, HRB_risk, default='Other')
try_df['Risk_Path'] = np.select(HRB_conditions, HRB_path, default='Other')
#try_df.to_excel('HRB_scored_v1.xlsx', index = True)

eda_working_df1_risk_assigned_v2 = try_df.copy(deep=True)

In [117]:
try_df = eda_working_df2.copy()

#code here
NHRB_conditions = [
    (try_df['salary'] <= 325) & (try_df['gender'] == 'M') & (try_df['livingperiod'] <= 1.5)
    & (try_df['financeprice_USD'] > 2585),
    (try_df['salary'] <= 325) & (try_df['gender'] == 'M') & (try_df['livingperiod'] <= 1.5)
    & (try_df['financeprice_USD'] <= 2585),
    (try_df['salary'] <= 325) & (try_df['gender'] != 'M') & (try_df['position'] == 'OWNER'),
    (try_df['salary'] <= 325) & (try_df['gender'] == 'M') & (try_df['livingperiod'] > 1.5)
    & (try_df['livingwith'] != 'SPOUSE'),
    (try_df['salary'] > 325) & (try_df['period'] > 30) & (try_df['livingperiod'] <= 31.5)
    & (try_df['salary'] <= 402.585),
    (try_df['salary'] <= 325) & (try_df['gender'] == 'M') & (try_df['livingperiod'] > 1.5)
    & (try_df['livingwith'] == 'SPOUSE'),
    (try_df['salary'] > 325) & (try_df['period'] <= 30) & (try_df['livingperiod'] <= 1.5),
    (try_df['salary'] <= 325) & (try_df['gender'] != 'M') & (try_df['position'] != 'OWNER')
    & (try_df['salary'] > 199),
    (try_df['salary'] > 325) & (try_df['period'] > 30) & (try_df['livingperiod'] <= 31.5)
    & (try_df['salary'] > 402.585),
    (try_df['salary'] > 325) & (try_df['period'] <= 30) & (try_df['livingperiod'] > 1.5)
    & (try_df['companytype'] == 'PRIVATE COMPANY (LESS 50 STAFFS)'),
    (try_df['salary'] <= 325) & (try_df['gender'] != 'M') & (try_df['position'] != 'OWNER')
    & (try_df['salary'] <= 199),
    (try_df['salary'] > 325) & (try_df['period'] <= 30) & (try_df['livingperiod'] > 1.5)
    & (try_df['companytype'] != 'PRIVATE COMPANY (LESS 50 STAFFS)'),
    (try_df['salary'] > 325) & (try_df['period'] > 30) & (try_df['livingperiod'] > 31.5)    
]

NHRB_risk = ['Very High Risk','Very High Risk','High Risk','High Risk',
             'Middle Risk','Middle Risk','Middle Risk','Middle Risk',
             'Low Risk','Low Risk','Low Risk','Low Risk',
             'Low Risk']

NHRB_path = ['Path 1', 'Path 2','Path 3','Path 4',
            'Path 5','Path 6','Path 7','Path 8',
            'Path 9','Path 10','Path 11','Path 12',
             'Path 13'
           ]

try_df['Risk_Grade'] = np.select(NHRB_conditions, NHRB_risk, default='Other')
try_df['Risk_Path'] = np.select(NHRB_conditions, NHRB_path, default='Other')
#try_df.to_excel('NHRB_scored_v1.xlsx', index = True)

eda_working_df2_risk_assigned_v2 = try_df.copy(deep=True)

In [118]:
df_graded_v2 = pd.concat([eda_working_df1_risk_assigned_v2, eda_working_df2_risk_assigned_v2], axis=0)

In [119]:
#branchcode
df_graded_v2['branchcode_grp'] = np.where(
    (df_graded_v2['branchcode'] == '9') | (df_graded_v2['branchcode'] == '10')
    | (df_graded_v2['branchcode'] == '15'),
    'HighRisk_branch', df_graded_v2['branchcode'])

df_graded_v2['branchcode_grp'] = np.where(
    (df_graded_v2['branchcode_grp'] != 'HighRisk_branch')
    , 'NonHighRisk_branch', df_graded_v2['branchcode_grp'])

df_graded_v2['branchcode'] = df_graded_v2['branchcode'].astype(int)

In [120]:
HRB_df = df_graded_v2[df_graded_v2.branchcode_grp == 'HighRisk_branch']
HRB_df_default = df_graded_v2[(df_graded_v2.branchcode_grp == 'HighRisk_branch') & (df_graded_v2.D3_within8m == 1)]
print(HRB_df.shape)
print(HRB_df_default.shape)

NHRB_df = df_graded_v2[df_graded_v2.branchcode_grp == 'NonHighRisk_branch']
NHRB_df_default = df_graded_v2[(df_graded_v2.branchcode_grp == 'NonHighRisk_branch') & (df_graded_v2.D3_within8m == 1)]
print(NHRB_df.shape)
print(NHRB_df_default.shape)

(3156, 25)
(433, 25)
(4309, 25)
(295, 25)


In [121]:
var_1 = 'branchcode'
var_2 = 'Risk_Grade'
HRB_cross_all = pd.crosstab(HRB_df[var_1], HRB_df[var_2], dropna=False)
HRB_cross_all.sort_index()

Risk_Grade,High Risk,Low Risk,Middle Risk,Very High Risk
branchcode,,,,
9,316,365,119,972
10,195,69,16,457
15,159,118,19,351


In [122]:
var_1 = 'branchcode'
var_2 = 'Risk_Grade'
HRB_cross_default = pd.crosstab(HRB_df_default[var_1], HRB_df_default[var_2], dropna=False)
HRB_cross_default.sort_index()

Risk_Grade,High Risk,Low Risk,Middle Risk,Very High Risk
branchcode,,,,
9,29,8,10,162
10,28,7,0,106
15,13,4,2,64


In [123]:
var_1 = 'branchcode'
var_2 = 'Risk_Grade'
NHRB_cross_all = pd.crosstab(NHRB_df[var_1], NHRB_df[var_2], dropna=False)
NHRB_cross_default = pd.crosstab(NHRB_df_default[var_1], NHRB_df_default[var_2], dropna=False)

with pd.ExcelWriter('NHRB_crosstab_v2 temp.xlsx', engine='openpyxl') as writer:
    NHRB_cross_all.to_excel(writer,sheet_name='all count', index = True)
    NHRB_cross_default.to_excel(writer,sheet_name='default', index = True)